# DHM Hydro Adjustment Workflow

This notebook demonstrates how to use the refactored `dhm-hydro-adjust` package to create hydrologically adjusted DTM rasters.

The workflow includes:
1. Filtering vector data by raster bounds
2. Sampling elevation data for line and horseshoe objects
3. Burning the adjustment objects into the DTM
4. Creating a hydro-adjusted DTM output

## Installation

First, install the package in development mode:

```bash
pip install -e .
```

Or install dependencies directly:

```bash
pip install gdal numpy scipy tqdm geopandas shapely
```

In [ ]:
# Import required libraries
import os
from pathlib import Path
import logging

# Import the hydro adjustment workflow
from hydroadjust import HydroAdjustWorkflow, create_hydro_adjusted_dtm

# Set up logging to see progress
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

print("DHM Hydro Adjust package imported successfully!")

## Configuration

Define your input files and output directory here. Modify these paths according to your data location.

In [ ]:
# === CONFIGURATION SECTION ===
# Modify these paths according to your data location

# Root directories
dtm_root = Path('D:/GeoILUN/data/Lemvig')  # Directory containing DTM data
correction_root = Path('D:/DK/tilpasninger')  # Directory containing correction vectors
output_root = Path('./output')  # Output directory for results

# Input files
dtm_raster = dtm_root / 'dtm_10.tif'  # Input DTM raster
horseshoe_file = correction_root / 'DHMHestesko.gpkg'  # Horseshoe vector data
line_file = correction_root / 'DHMLinje.gpkg'  # Line vector data

# Layer names in the vector files
horseshoe_layer = 'dhmhestesko'
line_layer = 'dhmlinje'

# Optional: Maximum sampling distance for horseshoe profiles (in map units)
# If None, will use half the diagonal pixel size of the input raster
max_sample_distance = None  # e.g., 0.1 for 0.1 meter sampling

print(f"DTM Raster: {dtm_raster}")
print(f"Horseshoe File: {horseshoe_file}")
print(f"Line File: {line_file}")
print(f"Output Directory: {output_root}")

## Method 1: Simple One-Step Workflow

The easiest way to run the complete workflow in a single function call:

In [ ]:
# Run the complete workflow in one step
try:
    result_dtm = create_hydro_adjusted_dtm(
        dtm_raster=dtm_raster,
        horseshoe_file=horseshoe_file,
        line_file=line_file,
        output_dir=output_root,
        horseshoe_layer=horseshoe_layer,
        line_layer=line_layer,
        max_sample_dist=max_sample_distance
    )
    
    if result_dtm:
        print(f"\n✅ Success! Hydro-adjusted DTM created: {result_dtm}")
    else:
        print("⚠️ No output created - no valid data found within raster bounds")
        
except Exception as e:
    print(f"❌ Error: {e}")

## Method 2: Step-by-Step Workflow

For more control over the process, you can run each step individually:

In [ ]:
# Initialize the workflow
workflow = HydroAdjustWorkflow(
    dtm_raster=dtm_raster,
    horseshoe_file=horseshoe_file,
    line_file=line_file,
    output_dir=output_root
)

print("Workflow initialized successfully!")
print(f"Output files will be created in: {workflow.output_dir}")

In [ ]:
# Step 1: Get raster bounds and filter vector data
print("Step 1: Filtering vector data by raster bounds...")

# Get the bounds of the DTM raster
bounds = workflow.get_raster_bounds()
print(f"Raster bounds: {bounds}")

# Filter vector data to only include features within the raster bounds
workflow.filter_vectors_by_bounds(
    horseshoe_layer=horseshoe_layer,
    line_layer=line_layer
)

print("✅ Vector filtering complete")

In [ ]:
# Step 2: Sample elevation for line objects
print("Step 2: Sampling elevation for line objects...")

if os.path.exists(workflow.lines_filtered):
    workflow.sample_line_z(
        input_lines=workflow.lines_filtered,
        output_lines=workflow.lines_with_z
    )
    print("✅ Line elevation sampling complete")
else:
    print("⚠️ No filtered line data found - skipping line sampling")

In [ ]:
# Step 3: Sample elevation for horseshoe objects
print("Step 3: Sampling elevation for horseshoe objects...")

if os.path.exists(workflow.hs_filtered):
    workflow.sample_horseshoe_z_lines(
        input_horseshoes=workflow.hs_filtered,
        output_lines=workflow.hs_with_z,
        max_sample_dist=max_sample_distance
    )
    print("✅ Horseshoe elevation sampling complete")
else:
    print("⚠️ No filtered horseshoe data found - skipping horseshoe sampling")

In [ ]:
# Step 4: Merge line files
print("Step 4: Merging line files...")

input_files = []
if os.path.exists(workflow.lines_with_z):
    input_files.append(workflow.lines_with_z)
if os.path.exists(workflow.hs_with_z):
    input_files.append(workflow.hs_with_z)

if input_files:
    workflow.merge_line_files(
        input_files=input_files,
        output_file=workflow.combined_lines
    )
    print(f"✅ Merged {len(input_files)} files into {workflow.combined_lines}")
else:
    print("⚠️ No line files to merge")

In [ ]:
# Step 5: Burn lines into DTM
print("Step 5: Burning lines into DTM...")

if os.path.exists(workflow.combined_lines):
    workflow.burn_lines_to_raster(
        lines_file=workflow.combined_lines,
        output_raster=workflow.hydro_dtm
    )
    print(f"✅ Hydro-adjusted DTM created: {workflow.hydro_dtm}")
else:
    print("⚠️ No combined lines file found - cannot burn into DTM")

## Results Summary

Let's check what files were created during the workflow:

In [ ]:
# Summary of output files
print("\n=== WORKFLOW RESULTS ===")
print(f"Output directory: {output_root}")
print("\nGenerated files:")

output_files = [
    ("Filtered horseshoes", workflow.hs_filtered),
    ("Filtered lines", workflow.lines_filtered),
    ("Horseshoes with Z", workflow.hs_with_z),
    ("Lines with Z", workflow.lines_with_z),
    ("Combined lines", workflow.combined_lines),
    ("Hydro-adjusted DTM", workflow.hydro_dtm)
]

for description, filepath in output_files:
    if os.path.exists(filepath):
        size_mb = os.path.getsize(filepath) / (1024 * 1024)
        print(f"✅ {description}: {filepath} ({size_mb:.2f} MB)")
    else:
        print(f"❌ {description}: {filepath} (not created)")

if os.path.exists(workflow.hydro_dtm):
    print(f"\n🎉 SUCCESS: Final hydro-adjusted DTM ready at {workflow.hydro_dtm}")
else:
    print(f"\n⚠️ WARNING: No final DTM was created")

## Alternative: Using Original CLI Scripts

If you prefer to use the original command-line scripts from within the notebook, you can still do so:

In [ ]:
# Alternative approach using the original CLI scripts
import subprocess

# Make sure the package is installed so the CLI commands are available
try:
    # Test if CLI commands are available
    result = subprocess.run(['sample_line_z', '--help'], capture_output=True, text=True)
    if result.returncode == 0:
        print("✅ CLI commands are available")
        
        # Example usage of CLI commands
        print("\nExample CLI usage:")
        print(f"sample_line_z {dtm_raster} {workflow.lines_filtered} {workflow.lines_with_z}")
        print(f"sample_horseshoe_z_lines {dtm_raster} {workflow.hs_filtered} {workflow.hs_with_z}")
        print(f"burn_line_z {workflow.combined_lines} {dtm_raster} {workflow.hydro_dtm}")
    else:
        print("❌ CLI commands not available. Install the package with: pip install -e .")
except FileNotFoundError:
    print("❌ CLI commands not found. Install the package with: pip install -e .")

## Next Steps

1. **Visualize Results**: Use your preferred GIS software to compare the original DTM with the hydro-adjusted DTM
2. **Quality Control**: Check the adjustment objects and verify they were burned correctly
3. **Integration**: Use the hydro-adjusted DTM in your hydrological modeling workflow

The refactored workflow makes it easy to:
- Run the complete process with sensible defaults
- Customize individual steps as needed
- Access intermediate results for quality control
- Integrate with other notebook-based workflows